In [ ]:
!wget https://mcauleylab.ucsd.edu/public_datasets/gdrive/googlelocal/review-Delaware_10.json.gz

In [ ]:
!wget https://mcauleylab.ucsd.edu/public_datasets/gdrive/googlelocal/meta-Delaware.json.gz

In [5]:
import gzip
import json
import pandas as pd
import re # Thư viện regex để làm sạch text
import itertools # Để đọc một phần của file lớn

# Hàm parse để đọc file .json.gz
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield json.loads(l)

print("Các thư viện đã được import và hàm parse đã sẵn sàng.")

Các thư viện đã được import và hàm parse đã sẵn sàng.


In [6]:
# --- Đọc file Review ---
# Giới hạn số lượng review để load cho nhanh, bạn có thể tăng sau
num_reviews_to_load = 100000
reviews_data = list(itertools.islice(parse('review-Delaware_10.json.gz'), num_reviews_to_load))
df_reviews_new = pd.DataFrame.from_records(reviews_data)

# --- Đọc file Metadata ---
metadata_data = list(parse('meta-Delaware.json.gz'))
df_meta_new = pd.DataFrame.from_records(metadata_data)

print("Đã đọc dữ liệu vào DataFrames thành công!")
print(f"Số review đã tải: {len(df_reviews_new)}")
print(f"Số địa điểm đã tải: {len(df_meta_new)}")

# Hiển thị 5 dòng đầu của mỗi DataFrame để kiểm tra
print("\n--- Dữ liệu Review (mẫu): ---")
print(df_reviews_new.head())

print("\n--- Dữ liệu Metadata (mẫu): ---")
print(df_meta_new.head())

Đã đọc dữ liệu vào DataFrames thành công!
Số review đã tải: 100000
Số địa điểm đã tải: 14706

--- Dữ liệu Review (mẫu): ---
                 user_id            name           time  rating  \
0  113797972931183350426  Heather Carper  1507258129698       5   
1  108911577609871083315  STACY CLAVETTE  1544802238870       5   
2  110147089361428367751       Zion Hood  1559123933643       5   
3  114714285192292470942     Diane Clark  1600736020990       5   
4  108986391413141739824         Eddie S  1554953350224       4   

                                                text  \
0  Lived here for 3 years and enjoyed it. Locatio...   
1  Nice complex and awesome staff.  Maintenance i...   
2  Good places for people to live my friend lives...   
3                                      great layout.   
4  Positives: Elevator, dog park and maintenance....   

                                                pics  resp  \
0  [{'url': ['https://lh5.googleusercontent.com/p...  None   
1           

In [7]:
# --- Chuẩn hóa DataFrame Review ---
# Chọn các cột hữu ích, bỏ bớt các cột không cần thiết như 'pics', 'resp'
df_reviews_new = df_reviews_new[['gmap_id', 'user_id', 'name', 'time', 'rating', 'text']]
# Đổi tên cột 'name' để tránh trùng với tên địa điểm trong metadata
df_reviews_new.rename(columns={'name': 'user_name'}, inplace=True)

# Xử lý các review rỗng (nếu có)
df_reviews_new.dropna(subset=['text'], inplace=True)

# --- Định nghĩa lại hàm clean_text từ Day 1 ---
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()  # Chuyển chữ thường
    text = re.sub(r'[^a-z\s]', '', text) # Loại bỏ ký tự đặc biệt, số...
    text = re.sub(r'\s+', ' ', text).strip() # Loại bỏ khoảng trắng thừa
    return text

# Áp dụng hàm làm sạch
df_reviews_new['text_clean'] = df_reviews_new['text'].apply(clean_text)

print("Đã chuẩn hóa và làm sạch DataFrame Review.")
print(df_reviews_new[['text', 'text_clean']].head())

# --- Chuẩn hóa DataFrame Metadata ---
# Chọn các cột hữu ích cho bài toán
df_meta_new = df_meta_new[['gmap_id', 'name', 'address', 'category', 'avg_rating', 'num_of_reviews', 'price']]
# Đổi tên cột 'name' để dễ phân biệt
df_meta_new.rename(columns={'name': 'place_name'}, inplace=True)

print("\nĐã chuẩn hóa DataFrame Metadata.")
print(df_meta_new.head())

Đã chuẩn hóa và làm sạch DataFrame Review.
                                                text  \
0  Lived here for 3 years and enjoyed it. Locatio...   
1  Nice complex and awesome staff.  Maintenance i...   
2  Good places for people to live my friend lives...   
3                                      great layout.   
4  Positives: Elevator, dog park and maintenance....   

                                          text_clean  
0  lived here for years and enjoyed it location t...  
1  nice complex and awesome staff maintenance is ...  
2  good places for people to live my friend lives...  
3                                       great layout  
4  positives elevator dog park and maintenance ne...  

Đã chuẩn hóa DataFrame Metadata.
                                 gmap_id  \
0   0x89b8b77c34771c5f:0xf768433b3a39763   
1  0x89c7aaafd9511449:0x6c168a2218e745da   
2  0x89c6ff8848b2bb41:0xe2d072c749a4cf8b   
3  0x89c6fd38d08c9aa3:0xd541b1a50e4d212c   
4  0x89c77ca685822e63:0xd3ab23a4294a

In [8]:
# Gộp hai DataFrame lại với nhau
df_combined = pd.merge(df_reviews_new, df_meta_new, on='gmap_id', how='left')

print("Gộp hai DataFrame thành công!")
print(f"Số dòng trong DataFrame kết hợp: {len(df_combined)}")
print("Kiểm tra các cột mới được thêm vào:")
print(df_combined.head())

Gộp hai DataFrame thành công!
Số dòng trong DataFrame kết hợp: 58289
Kiểm tra các cột mới được thêm vào:
                                gmap_id                user_id  \
0  0x89b8b77c34771c5f:0xf768433b3a39763  113797972931183350426   
1  0x89b8b77c34771c5f:0xf768433b3a39763  113797972931183350426   
2  0x89b8b77c34771c5f:0xf768433b3a39763  108911577609871083315   
3  0x89b8b77c34771c5f:0xf768433b3a39763  108911577609871083315   
4  0x89b8b77c34771c5f:0xf768433b3a39763  110147089361428367751   

        user_name           time  rating  \
0  Heather Carper  1507258129698       5   
1  Heather Carper  1507258129698       5   
2  STACY CLAVETTE  1544802238870       5   
3  STACY CLAVETTE  1544802238870       5   
4       Zion Hood  1559123933643       5   

                                                text  \
0  Lived here for 3 years and enjoyed it. Locatio...   
1  Lived here for 3 years and enjoyed it. Locatio...   
2  Nice complex and awesome staff.  Maintenance i...   
3  Nice c

In [9]:
# Lưu DataFrame đã gộp thành file CSV
output_filename = 'ucsd_delaware_reviews_combined.csv'
df_combined.to_csv(output_filename, index=False)

print(f"\nĐã lưu dữ liệu đã xử lý vào file '{output_filename}'.")
print("Bây giờ bạn đã sẵn sàng để chuyển sang Day 2 với bộ dữ liệu nâng cấp này!")


Đã lưu dữ liệu đã xử lý vào file 'ucsd_delaware_reviews_combined.csv'.
Bây giờ bạn đã sẵn sàng để chuyển sang Day 2 với bộ dữ liệu nâng cấp này!
